In [1]:
import itertools
from Utility import *

# for the plotting
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np

In [2]:
import argparse
import json
import os

TEST_RESULTS_DIRECTORY = './test_results/'
GENDER_DIFFERENCES_RESULTS_DIRECTORY = './gender_differences_results/'
ABSOLUTE_SCORE_RESULTS_DIRECTORY = './absolute_score_results/'
DATASET_NAME = 'Wikigender'
RELATIONS = ['spouse', 'birthdate', 'birthplace', 'hypernym']
RAW_METRICS = ['num_correct', 'num_predicted', 'num_actual']
METRICS = ['f1_score', 'precision', 'recall']
ENCODERS = ['pcnn', 'cnn', 'rnn', 'birnn']
SELECTORS = ['att', 'ave']



def readFromJsonFile(infile_name):
    with open(infile_name, 'r') as infile:
        return json.load(infile)
    return ""
def writeToJsonFile(data, outfile_name, prettify=False):
    with open(outfile_name, 'w') as outfile:
        if(prettify):
            json.dump(data, outfile, indent=4, sort_keys=True)
        else:
            json.dump(data, outfile)

def  getTestFiles(dir_name, model_name):
    '''
    :param dir_name:
    :param model_name:
    :return: returns (female test file, male test file) tuple
    '''


    test_suffix_female = "_FEmale_pred.json"
    test_suffix_male = "_male_pred.json"

    female_file = readFromJsonFile(os.path.join(dir_name, model_name + test_suffix_female))
    male_file = readFromJsonFile(os.path.join(dir_name, model_name + test_suffix_male))

    return (male_file, female_file)

def getModelName(args, dataset_name, encoder, selector):
    name_suffix = getNameSuffix(args)
    if args.debiased_embeddings:
        name_suffix += "_DE"
    else:
        name_suffix += "_NoDE"

    if name_suffix == '':
        return dataset_name + "_" + encoder + "_" + selector
    elif name_suffix == '_DE' or name_suffix == '_NoDE':
        return dataset_name + "_" + encoder + "_" + selector + name_suffix
    else:
        return dataset_name + "_" + encoder + "_" + selector + "_" + name_suffix

def getNameSuffix(args):
    '''
    :param args: some argparser containing arguments:
                    name_anonymize: boolean, true if model trained on name anonned data
                    equalized_gender_mentions:
                    gender_swap:
    :return:
    '''
    name_suffix = ''

    if not args.name_anonymize and not args.equalized_gender_mentions and not args.gender_swap:
        # then we don't want to use any suffix or anything; we're using the regular dataset
        return ''

    if args.name_anonymize:
        name_suffix += "_NA"
    else:
        name_suffix += "_NoNA"
    if args.equalized_gender_mentions:
        name_suffix += "_Eq"
    else:
        name_suffix += "_NoEq"
    if args.gender_swap:
        name_suffix += "_GS"
        if args.swap_names:
            name_suffix += "_NS"
        else:
            name_suffix += "_NoNS"
    else:
        name_suffix += "_NoGS"

    return name_suffix

def getModelNameSimulateArgs(equalized_gender_mentions=False, gender_swapped=False, name_anonymized=False, debiased_embeddings=False, encoder='pcnn', selector='att'):
    parser = argparse.ArgumentParser()
    parser.add_argument('--dataset', nargs='?', default='Wikigender')
    parser.add_argument('--encoder', nargs='?', default='pcnn')
    parser.add_argument('--selector', nargs='?', default='att')
    parser.add_argument("--gender_swap", "-gs", action="store_true")
    parser.add_argument("--equalized_gender_mentions", "-egm", action="store_true")
    parser.add_argument("--swap_names", "-sn", action="store_true")
    parser.add_argument("--name_anonymize", "-na", action="store_true")
    parser.add_argument("--debiased_embeddings", "-de", action="store_true")
    args = parser.parse_args()

    args.name_anonymize = name_anonymized
    args.gender_swap = gender_swapped
    args.equalized_gender_mentions = equalized_gender_mentions
    args.debiased_embeddings = debiased_embeddings
    args.encoder = encoder
    args.selector = selector

    # get the model name
    model_name = getModelName(args, "Wikigender", args.encoder, args.selector)
    return model_name

In [3]:
def calc_fbeta_score(num_correct, num_predicted, num_actual, beta):
    precision = num_correct / num_predicted
    recall = num_correct / num_actual

    beta_squared = beta * beta

    return (1 + beta_squared) * ((precision * recall) / ((((beta_squared) * precision)) + recall))

def calc_pps_score(alpha, abs_f_score, parity_f_score, numt_classes):
    first_term = alpha * (1/len(RELATIONS)) * abs_f_score
    second_term = (1 - alpha) * (1/len(RELATIONS))  * (1 - (parity_f_score)/num_classes)

    return first_term + second_term

In [4]:
def get_raw_results(male_results, female_results):
    raw_metric_abs_scores = dict()
    for relation in RELATIONS:
        raw_metric_abs_scores[relation] = dict()

        for raw_metric in RAW_METRICS:
            if raw_metric in raw_metric_abs_scores[relation]:
                raw_metric_abs_scores[relation][raw_metric]['total'] += male_results[relation][raw_metric] + \
                                                              female_results[relation][raw_metric]
                raw_metric_abs_scores[relation][raw_metric]['male'] += male_results[relation][raw_metric]
                raw_metric_abs_scores[relation][raw_metric]['female'] += female_results[relation][raw_metric]
            else:
                raw_metric_abs_scores[relation][raw_metric] = dict()

                raw_metric_abs_scores[relation][raw_metric]['total'] = male_results[relation][raw_metric] + \
                                                             female_results[relation][raw_metric]
                raw_metric_abs_scores[relation][raw_metric]['male'] = male_results[relation][raw_metric]
                raw_metric_abs_scores[relation][raw_metric]['female'] = female_results[relation][raw_metric]
                
    return raw_metric_abs_scores

In [5]:
def get_pps_scores(male_results, female_results, alpha_vals, beta_vals):
    '''
    this function obtains the pps scores for a LIST of beta values and alpha values
    :param male_results: 
    :param female_results: 
    :param beta_vals: a list giving beta values
    :param alpha_vals: a list giving alpha values
    :return: A matrix of pps scores where pps[i][j] corresponds to beta_vals[j] and alpha_vals[i], a dictionary which can be written toa  file giving [absolute][beta_val][alpha_val] = absolute_val or [parity][beta_val]{alhpa_val} = parity_val
    '''
    
    pps_scores = list()
    pps_dict = dict()
    
    '''get the desired raw results (num_correct, num_actual, num_predicted for each relation and male, female predictions for each relation'''
    raw_results = get_raw_results(male_results, female_results)
    
    for alpha_index in range(len(alpha_vals)):
        alpha = alpha_vals[alpha_index]

        # update structures for this round of alpha vals
        pps_scores.append(list())
        pps_dict['alpha={}'.format(alpha)] = dict()

        for beta_index in range(len(beta_vals)):
            beta = beta_vals[beta_index]

            # update dict for this round of beta vals
            pps_dict['alpha={}'.format(alpha)]['beta={}'.format(beta)] = dict()

            # get the sume of these over the relations
            parity_fbeta = 0
            absolute_fbeta = 0
            for relation in RELATIONS:
                male_fbeta = calc_fbeta_score(raw_results[relation]['num_correct']['male'],
                                             raw_results[relation]['num_predicted']['male'],
                                             raw_results[relation]['num_actual']['male'],
                                             beta)
                female_fbeta = calc_fbeta_score(raw_results[relation]['num_correct']['female'],
                                               raw_results[relation]['num_predicted']['female'],
                                               raw_results[relation]['num_actual']['female'],
                                               beta)

                # we're summing these over all the relations!
                parity_fbeta += abs(male_fbeta - female_fbeta)

                absolute_fbeta += calc_fbeta_score(raw_results[relation]['num_correct']['total'],
                                                  raw_results[relation]['num_predicted']['total'],
                                                  raw_results[relation]['num_actual']['total'],
                                                  beta)
                
            # now, calculate pps for this configuration
            pps_score = calc_pps_score(alpha, absolute_fbeta, parity_fbeta, 2)

            # add to the data structures
            pps_scores[alpha_index].append(pps_score)
            pps_dict['alpha={}'.format(alpha)]['beta={}'.format(beta)]['absolute'] = absolute_fbeta
            pps_dict['alpha={}'.format(alpha)]['beta={}'.format(beta)]['parity'] = parity_fbeta
            pps_dict['alpha={}'.format(alpha)]['beta={}'.format(beta)]['pps_score'] = pps_score


    return pps_scores, pps_dict

In [6]:
def get_pps_scores_matplotlib(male_results, female_results, alpha_vals, beta_vals):
    '''
    this function obtains the pps scores for a LIST of beta values and alpha values
    :param male_results:
    :param female_results:
    :param beta_vals: a list giving beta values
    :param alpha_vals: a list giving alpha values
    :return: A matrix of pps scores where pps[i][j] corresponds to beta_vals[j] and alpha_vals[i], a dictionary which can be written toa  file giving [absolute][beta_val][alpha_val] = absolute_val or [parity][beta_val]{alhpa_val} = parity_val
    '''

    pps_scores = list()
    pps_dict = dict()

    '''get the desired raw results (num_correct, num_actual, num_predicted for each relation and male, female predictions for each relation'''
    raw_results = get_raw_results(male_results, female_results)

    for alpha_index in range(len(alpha_vals)):
        alpha = alpha_vals[alpha_index]

        # update structures for this round of alpha vals
        pps_scores.append(list())
        pps_dict['alpha={}'.format(alpha)] = dict()

        for beta_index in range(len(beta_vals)):
            beta = beta_vals[beta_index]

            # update dict for this round of beta vals
            pps_dict['alpha={}'.format(alpha)]['beta={}'.format(beta)] = dict()

            # get the sume of these over the relations
            parity_fbeta = 0
            absolute_fbeta = 0
            for relation in RELATIONS:
                male_fbeta = calc_fbeta_score(raw_results[relation]['num_correct']['male'],
                                              raw_results[relation]['num_predicted']['male'],
                                              raw_results[relation]['num_actual']['male'],
                                              beta)
                female_fbeta = calc_fbeta_score(raw_results[relation]['num_correct']['female'],
                                                raw_results[relation]['num_predicted']['female'],
                                                raw_results[relation]['num_actual']['female'],
                                                beta)

                # we're summing these over all the relations!
                parity_fbeta += abs(male_fbeta - female_fbeta)

                absolute_fbeta += calc_fbeta_score(raw_results[relation]['num_correct']['total'],
                                                   raw_results[relation]['num_predicted']['total'],
                                                   raw_results[relation]['num_actual']['total'],
                                                   beta)

            # now, calculate pps for this configuration
            pps_score = calc_pps_score(alpha, absolute_fbeta, parity_fbeta, 2)

            # add to the data structures
            pps_scores[alpha_index].append(pps_score)
            pps_dict['alpha={}'.format(alpha)]['beta={}'.format(beta)]['absolute'] = absolute_fbeta
            pps_dict['alpha={}'.format(alpha)]['beta={}'.format(beta)]['parity'] = parity_fbeta
            pps_dict['alpha={}'.format(alpha)]['beta={}'.format(beta)]['pps_score'] = pps_score

    return pps_scores, pps_dict

In [7]:
def get_pps_scores_for_model(combo, alpha_vals, beta_vals):
    '''

    :param combo: gives the debiasing configuration of the model so we know what its name is
    :param alpha_vals: alpha values for pps score
    :param beta_vals: beta values for pps score
    :return: pps scores for this model
    '''
    # get results files for this model name
    model_name = getModelNameSimulateArgs(combo[0], combo[1], combo[2], combo[3])
    male_results, female_results = getTestFiles(TEST_RESULTS_DIRECTORY, model_name)

    # get the pps results
    pps_scores, pps_dict = get_pps_scores(male_results, female_results, alpha_vals, beta_vals)

    # write the dict object to a file
    outfile_path = os.path.join('pps_scores', model_name + "pps_scores.json")
    writeToJsonFile(pps_dict, outfile_path, True)

    return pps_scores

In [8]:
def plot_3d(X, Y, male_results, female_results):
    fig = plt.figure()
    ax = fig.gca(projection='3d')

    # Make data.
    X, Y = np.meshgrid(X, Y)
    R = np.sqrt(X ** 2 + Y ** 2)
    Z = np.sin(R)

    pps_scores, _ = get_pps_scores(male_results, female_results, np.ravel(X), np.ravel(Y))

    Z = np.array([np.array(item) for item in pps_scores])
    Z = Z.reshape(X.shape)


    # Plot the surface.
    surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                           linewidth=0, antialiased=False)

    # Customize the z axis.
    ax.set_zlim(-1.01, 1.01)
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

    # Add a color bar which maps values to colors.
    fig.colorbar(surf, shrink=0.5, aspect=5)

    plt.show()

In [9]:
items = [True, False]
true_false_combos = [list(i) for i in itertools.product(items, repeat=4)]

for combo in true_false_combos:
    try:
        get_pps_scores_for_model(combo, [0.5], [1])
    except FileNotFoundError as e:
        print(e)
        continue

# 3D plot the values for the PPS scores
# alpha_vals = np.arange(0,1.10,0.10) # get alpha values with step value of 0.01
# beta_vals = np.arange(0,11,1)
# pps_scores = get_pps_scores_for_model([False, True, False, False], alpha_vals, beta_vals)
# z_vals = np.asarray(pps_scores)

# model_name = getModelNameSimulateArgs(combo[0], combo[1], combo[2], combo[3])
# male_results, female_results = getTestFiles(TEST_RESULTS_DIRECTORY, model_name)
# plot_3d(alpha_vals, beta_vals, male_results, female_results)

usage: ipykernel_launcher.py [-h] [--dataset [DATASET]] [--encoder [ENCODER]]
                             [--selector [SELECTOR]] [--gender_swap]
                             [--equalized_gender_mentions] [--swap_names]
                             [--name_anonymize] [--debiased_embeddings]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/tsun/Library/Jupyter/runtime/kernel-e7d4ef51-e6ab-4330-9992-3e90c0c65074.json


SystemExit: 2

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
